## Task 1 - Third-Order Letter Approximation Model

The below code takes in all of the txt files in texts/, filtering out special & numerical characters, then counting how often each sequence of three characters appears and compiles it into a dictionary.

<hr>

In [ ]:
import os
import re

texts = []

for title in os.scandir("texts"): # [2]
    if((title.name.find(".txt")) != -1): # Only attempt to open + add the text file if it's a .txt
        fileContent = open(title) # [1]
        texts.append(fileContent.read()) # Add it to the texts list.
        fileContent.close()

for t in texts:
    print(sanitizeText(t))

In [16]:
def sanitizeText(text):
    preamble = " ***" # Ending of the preamble
    postamble = "*** END OF " # Beginning of the postamble.

    # [3] Strips out the preamble and postamble from a text by
    # creating a string in between the preamble and postamble variables.
    sanitizedText = (t[t.index(preamble)+len(preamble):t.index(postamble)]) 
    sanitizedText = re.sub("[^a-zA-Z\\s.]", "", sanitizedText) # [4]

    return sanitizedText.upper()

## Task 2 - Third-Order Letter Approximation Generation

<hr>

## Task 3 - Analyze The Model

<hr>

## Task 4 - Export Model

<hr>

## References

- [1] - Reading text files in Python: https://www.w3schools.com/python/python_file_open.
- [2] - Using Python's OS Module: https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
- [3] - Finding a string between two substrings (used for removing Pre/Postamble): https://stackoverflow.com/a/51456576
- [4] - Using regex in Python: https://www.w3schools.com/python/python_regex.asp
- 
- 
- 

<hr>